In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
import torch
import torchvision

# Para simplificar la transformación
from torchvision import transforms as T
from torchvision import models
from torchvision import datasets

from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split

In [3]:
import sys
from tqdm import tqdm
import time
import copy

In [4]:
import timm
from timm.loss import LabelSmoothingCrossEntropy

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:

def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir)
    return all_data.classes


In [8]:
def get_data_loader(data_dir, batch_size, train=False):
    if train:
        transform = T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.25),
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            T.RandomErasing(p=0.2, value='random')
        ])
        train_data = datasets.ImageFolder(
            os.path.join(data_dir, "train/"),
            transform=transform)
        train_loader = DataLoader(
            train_data,
            batch_size=batch_size,
            shuffle=True,
            num_workers=4)
        return train_loader, len(train_data)
    else:
        transform = T.Compose([
            
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        val_data = datasets.ImageFolder(
            os.path.join(data_dir, "valid/"),
            transform=transform)
        val_loader = DataLoader(
            val_data,
            batch_size=batch_size,
            shuffle=True,
            num_workers=4)
        test_data = datasets.ImageFolder(
            os.path.join(data_dir, "test/"),
            transform=transform)
        test_loader = DataLoader(
            test_data,
            batch_size=batch_size,
            shuffle=True,
            num_workers=4)
        return val_loader, test_loader, len(val_data), len(test_data)

In [9]:
dataset_path = './Images/pixeled/'

In [10]:
(train_loader, train_data_len)= get_data_loader(dataset_path,
                                                128,
                                                train=True)
(val_loader, test_loader, val_data_len, test_data_len) = get_data_loader(dataset_path,
                                                                           32, 
                                                                           train=False)

In [11]:
classes = get_classes("./Images/pixeled/train/")
print(classes, len(classes))

['pixeled'] 1


In [12]:
dataloaders={
    "train": train_loader,
    "val": val_loader
}

dataset_sizes={
    "train": train_data_len,
    "val": val_data_len
}

In [13]:
print(len(train_loader), len(val_loader), len(test_loader))

458 22 22


In [14]:
print(train_data_len, val_data_len, test_data_len)

58600 700 700


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [16]:
model = torch.hub.load('facebookresearch/deit:main',
                       'deit_tiny_patch16_224',
                       pretrained = True)

Using cache found in C:\Users\kpm_1/.cache\torch\hub\facebookresearch_deit_main


In [22]:
model

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=192, out_features=576, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=192, out_features=192, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=192, out_features=768, bias=True)
        (act): GELU(approximate=none)
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=768, out_features=192, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (ls2): Ide

In [17]:
for param in model.parameters():
    param.requires_grad = False

n_inputs = model.head.in_features
model.head = nn.Sequential(
    nn.Linear(n_inputs, 512),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(512, 2)
)

model = model.to(device)
print(model.head)

Sequential(
  (0): Linear(in_features=192, out_features=512, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=512, out_features=2, bias=True)
)


In [18]:
criterion = LabelSmoothingCrossEntropy()
criterion = criterion.to(device)

optimizer = optim.Adam(model.head.parameters(), lr=0.001)

In [19]:
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer,
                                             step_size=3,
                                             gamma=0.97)

In [20]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
#     since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print( f'Epoch {epoch}/{num_epochs - 1}')
        print('-'*10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0.0
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) ## para obtener el ACC
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step() #paso al final de la época
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, 
                                                       epoch_loss,
                                                       epoch_acc))
            
#             Revisar porque a veces la mejor puntuación puede 
#             darse al inicio y no necesariamente es la mejor opción 
#             (puede haber overfitting)
#             print('model.state_dict() ->', model.state_dict())
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print('****end phase****')
#     time_elapsed = time.time - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed//60, time_elapsed%60))
    print('Best Val Acc: {:.4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model

In [21]:
model_fit = train_model(model, criterion, optimizer, exp_lr_scheduler)

Epoch 0/9
----------


100%|████████████████████████████████████████████████████████████████████████████████| 458/458 [01:21<00:00,  5.65it/s]


train Loss: 0.2052 Acc: 0.9981


100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:03<00:00,  5.81it/s]


val Loss: 0.1989 Acc: 1.0000
****end phase****
Epoch 1/9
----------


...

100%|████████████████████████████████████████████████████████████████████████████████| 458/458 [01:28<00:00,  5.18it/s]


train Loss: 0.1988 Acc: 1.0000


100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:03<00:00,  5.87it/s]


val Loss: 0.1988 Acc: 1.0000
****end phase****


TypeError: unsupported operand type(s) for -: 'builtin_function_or_method' and 'float'